# An example of creating coefficients using pyEXP
This example assumes that you have run the `EXP/examples/Better` simulation.  But this notebook could be adapted for any simulation you like.

We begin by importing `pyEXP` and friends and setting the working directory.

In [1]:
import os
import yaml
import pyEXP

# I have the Better run here; obviously another use will want to
# change this a directory containing some snapshots of their own
#
os.chdir('/home/weinberg/Nbody/Better')

## Create the basis
We'll only do the halo coefficients in this simple example.  The cylindrical coefficients would procede similarly.  See the `sample_basis` notebook for an example of creating the cylindrical basis.

In [2]:
# Get the basis config
#
yaml_config = ""
with open('basis.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    yaml_config = yaml.dump(config)

# Alternatively, you could construct this on the fly, e.g.
bconfig = """
---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  rs: 0.0667
  modelname: SLGridSph.model
...
"""
print('-'*60)
print('Read from file')
print('-'*60)
print(yaml_config)
print('-'*60)
print('Constructed from string')
print('-'*60)
print(bconfig)
print('-'*60)

# Construct the basis instance
#
basis   = pyEXP.basis.Basis.factory(yaml_config)

------------------------------------------------------------
Read from file
------------------------------------------------------------
id: sphereSL
parameters:
  Lmax: 4
  modelname: SLGridSph.model
  nmax: 10
  numr: 2000
  rmax: 1.95
  rmin: 0.0001
  rs: 0.0667

------------------------------------------------------------
Constructed from string
------------------------------------------------------------

---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  rs: 0.0667
  modelname: SLGridSph.model
...

------------------------------------------------------------
---- SLGridSph::read_cached_table: trying to read cached table . . .
---- SLGridSph::read_cached_table: Success!!


## Creating a particle reader
Now that we have a basis, we can use it to create coefficients from the particle snapshots.  `pyEXP` uses a `ParticleReader` object for that.

The first step is to hand off the files that comprise a snapshot for every time slice.  The `ParticleReader` provides a helper function for that.   There are two helper functions: `parseFileList` and `parseStringList`.  The first reads a list from a file and the second takes a list.  Otherwise they are the same.  The file names in the list are assumed to end with a snapshot index and an optional part index.  For example, if you have single files per snapshot, the list might look like: `myrun.00000`, `myrun.00001`, etc.  If you have multiple files per snapshot, they will look something like `myrun.00000_0001`, `myrun.00000_0002`, `myrun.00001_0000`, `myrun.00001_0001`, etc.

Here is the call for a file:

In [3]:
# Construct batches of files the particle reader.  One could use the
# parseStringList to create batches from a vector/list of files.  NB:
# a std::vector in C++ becomes a Python.list and vice versa
#
batches = pyEXP.read.ParticleReader.parseFileList('file.list', '')

We now iterate the `batches` created by the file parser to create the coefficients.   For each batch we create a new reader and pass the reader to the basis instance.  The `basis.createFromReader` member creates and returns the coefficients.  The coefficients are added to a coefficient container called `coefs`.  Note: on the first call `coefs=None` so a new container is created on the first time through.

In [4]:
# This will contain the coefficient container, need to start will a
# null instance to trigger construction
#
coefs   = None

for group in batches:

    print("file group is", group)

    # Make the reader for the desired type.  One could probably try to
    # do this by inspection but that's another project.
    #
    reader = pyEXP.read.ParticleReader.createReader('PSPout', group, 0, False);

    # Print the type list
    #
    print('The component names are:', reader.GetTypes())

    compname = 'dark halo'
    reader.SelectType(compname)
    print('Selected', compname)

    print('Call createFromReader at Time', reader.CurrentTime(), 'for', reader.CurrentNumber(), 'particles')

    coef = basis.createFromReader(reader)
    print("Created coef")

    # We need this stupid idiom here because None is not mapping to a
    # null pointer.  There is probably a way to do this.  Suggestions
    # anyone?
    #                          This is optional---+
    #                                             |
    if coefs is None:           #                 v
        coefs = pyEXP.coefs.Coefs.makecoefs(coef, compname)
    else:
        coefs.add(coef)

    print('Added coef')
    print('-'*60)

print('\nCompleted the file group list\n')

print('The coefficient time list is', coefs.Times())

file group is ['OUT.run0.00000']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.0 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00001']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.005000000000000004 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00002']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.010000000000000007 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00003']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.015000000000000012 for 100000 particles
Created coef
Added coef
----------------------------------

Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00033']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.16499999999998877 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00034']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.16999999999998822 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00035']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.17499999999998767 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00036']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0

Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00065']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.32500000000000445 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00066']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.3300000000000061 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00067']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.3350000000000078 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00068']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.3

## Using a FieldGenerator
Now that we have our new coefficients, we can use the `FieldGenerator` to view the BFE representation of the underlying fields.  Here is an example:

In [5]:
# Now try some slices for rendering
#

times = coefs.Times()[0:3]
pmin  = [-1.0, -1.0, 0.0]
pmax  = [ 1.0,  1.0, 0.0]
grid  = [  40,   40,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

print("We now have the following [time field] pairs")
for v in surfaces:
    print('-'*40)
    for u in surfaces[v]:
        print("{:8.4f}  {}".format(v, u))

print("\nHere is the first one:")
for v in surfaces:
    for u in surfaces[v]:
        print('-'*40)
        print('----', u)
        print('-'*40)
        print(surfaces[v][u])
    break

We now have the following [time field] pairs
----------------------------------------
  0.0050  d
  0.0050  d0
  0.0050  d1
  0.0050  dd
  0.0050  fp
  0.0050  fr
  0.0050  ft
  0.0050  p
  0.0050  p0
  0.0050  p1
----------------------------------------
  0.0100  d
  0.0100  d0
  0.0100  d1
  0.0100  dd
  0.0100  fp
  0.0100  fr
  0.0100  ft
  0.0100  p
  0.0100  p0
  0.0100  p1
----------------------------------------
  0.0150  d
  0.0150  d0
  0.0150  d1
  0.0150  dd
  0.0150  fp
  0.0150  fr
  0.0150  ft
  0.0150  p
  0.0150  p0
  0.0150  p1

Here is the first one:
----------------------------------------
---- d
----------------------------------------
[[0.01461391 0.01571142 0.01685744 ... 0.01694855 0.01567789 0.01448951]
 [0.01578225 0.01701935 0.01832261 ... 0.01833056 0.01689233 0.01556032]
 [0.01703135 0.01843099 0.0199185  ... 0.01981544 0.01818381 0.01668729]
 ...
 [0.01620399 0.01761632 0.01916323 ... 0.01926629 0.0178635  0.01654805]
 [0.01514537 0.01640853 0.01777834 ...

These could be make into images and so forth.   We'll do this in another example notebook.

## Saving the coefficients

At this point, it makes sense to save the coefficients that you have just created.  This is sone with the following call:

In [6]:
coefs.WriteH5Coefs('test_better')

HDF5-DIAG: Error detected in HDF5 (1.10.7) MPI-process 0:
  #000: ../../../src/H5A.c line 285 in H5Acreate2(): unable to create attribute
    major: Attribute
    minor: Unable to initialize object
  #001: ../../../src/H5Aint.c line 165 in H5A__create(): attribute already exists
    major: Attribute
    minor: Object already exists
Unable to create the attribute "geometry": (Attribute) Object already exists


We now have a EXP HDF5 coefficient file called `test_better.h5`.    As an example, let's try reading the newly created file into another coefficient container, `coefs2`.  The container has a member function called `CompareStanzas` which will check on the contents.  Let's do it.

In [7]:
# Now try reading it in
#
coefs2 = pyEXP.coefs.Coefs.factory('test_better.h5')
print("Type is", coefs2.getGeometry())

# Now compare with the original
#
coefs2.CompareStanzas(coefs)

Type is sphere


False

Times are the same, now checking parameters at each time
Parameters are the same, now checking coefficients
Coefficient (4, 2)  (0.000372975,0.00346837) != (0.000372975,0.00346837)
Coefficient (5, 1)  (0.00259255,-0.00527381) != (0.00259255,-0.00527381)
Coefficient (7, 5)  (0.000219287,-0.000659943) != (0.000219287,-0.000659943)
Coefficient (7, 6)  (0.000863475,-2.103e-06) != (0.000863475,-2.103e-06)
Coefficient (8, 1)  (-0.00609356,-0.00985978) != (-0.00609356,-0.00985978)
Coefficient (9, 8)  (0.00117133,-0.000851888) != (0.00117133,-0.000851888)
Coefficient (13, 7)  (-8.24983e-05,-0.00569963) != (-8.24983e-05,-0.00569963)
Coefficient (5, 4)  (-0.00127168,0.00355475) != (-0.00127168,0.00355475)
Coefficient (8, 1)  (-0.00536253,-0.00980645) != (-0.00536253,-0.00980645)
Coefficient (9, 5)  (-0.00320046,-0.00161583) != (-0.00320046,-0.00161583)
Coefficient (9, 6)  (0.0013027,0.000585304) != (0.0013027,0.000585304)
Coefficient (9, 9)  (-0.000989662,-0.00351378) != (-0.000989662,-0.0035137

Coefficient (13, 9)  (0.000225262,-0.00194194) != (0.000225262,-0.00194194)
Coefficient (4, 0)  (-0.00693122,0.00651527) != (-0.00693122,0.00651527)
Coefficient (9, 2)  (0.00488948,-0.000295958) != (0.00488948,-0.000295958)
Coefficient (12, 0)  (0.00251636,1.44787e-05) != (0.00251636,1.44787e-05)
Coefficient (13, 1)  (8.69849e-05,0.000525455) != (8.69849e-05,0.000525455)
Coefficient (13, 4)  (0.00335594,-0.00213626) != (0.00335594,-0.00213626)
Coefficient (14, 7)  (-0.00133575,0.00144915) != (-0.00133575,0.00144915)
Coefficient (2, 1)  (0.00369842,0.0114794) != (0.00369842,0.0114794)
Coefficient (2, 8)  (-0.00038595,-0.00876316) != (-0.00038595,-0.00876316)
Coefficient (4, 1)  (-0.000606472,0.000812969) != (-0.000606472,0.000812969)
Coefficient (7, 8)  (0.00256438,-0.000787255) != (0.00256438,-0.000787255)
Coefficient (12, 7)  (0.00195087,-7.58665e-05) != (0.00195087,-7.58665e-05)
Coefficient (13, 2)  (-0.00121389,0.00127637) != (-0.00121389,0.00127637)
Coefficient (13, 6)  (0.00254214

This member function will print differences.  No differenced should be printed, of course.